In [9]:
import zipfile
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 数据概况

## 特征字段
|Field|Description|备注|
|--|--|--|
|SaleID|	交易ID|唯一编码|
|name|	汽车交易名称|已脱敏|
|regDate|	汽车注册日期|例如20160101，2016年01月01日|
|model|	车型编码|已脱敏，**原数据格式设置错误，应取整**|
|brand|	汽车品牌|已脱敏|
|bodyType|	车身类型|豪华轿车：0，微型车：1，厢型车：2，大巴车：3，敞篷车：4，双门汽车：5，商务车：6，搅拌车：7|
|fuelType|	燃油类型|汽油：0，柴油：1，液化石油气：2，天然气：3，混合动力：4，其他：5，电动：6|
|gearbox|	变速箱|手动：0，自动：1|
|power|	发动机功率|范围 [ 0, 600 ]|
|kilometer|	汽车已行驶公里|单位万km|
|notRepairedDamage|	汽车有尚未修复的损坏|是：0，否：1|
|regionCode|	地区编码|已脱敏|
|seller|	销售方|个体：0，非个体：|
|offerType|	报价类型|提供：0，请求：1|
|creatDate|	汽车上线时间|即开始售卖时间|
|**price**|二手车交易价格|（预测目标）|
|v0-14|	匿名特征|包含v0-14在内15个匿名特征|

## 评价指标
$$MAE=\frac{\sum_{i=1}^{n}|y_i-\hat{y}_i|}{n}$$

## 总结
本题是一个回归，特征有一些是数值特征、类型特征、时间特征。

# 数据准备

## 数据导入

In [14]:
def extract_data(fname):
    with zipfile.ZipFile(fname) as zfile:
        data_file = zfile.open(zfile.namelist()[0])
        data = pd.read_csv(data_file, sep=" ", index_col=0)
    return data

In [15]:
data_dir = r"data/"
fname_testA = os.path.join(data_dir, "used_car_testA_20200313.zip")
data_testA = extract_data(fname_testA)
fname_train = os.path.join(data_dir, "used_car_train_20200313.zip")
data_train = extract_data(fname_train)

## 数据概览

In [17]:
data_train.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,0.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,0.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [16]:
data_testA.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
150000,66932,20111212,222.0,4,5.0,1.0,1.0,313,15.0,0.0,...,0.264405,0.121800,0.070899,0.106558,0.078867,-7.050969,-0.854626,4.800151,0.620011,-3.664654
150001,174960,19990211,19.0,21,0.0,0.0,0.0,75,12.5,1.0,...,0.261745,0.000000,0.096733,0.013705,0.052383,3.679418,-0.729039,-3.796107,-1.541230,-0.757055
150002,5356,20090304,82.0,21,0.0,0.0,0.0,109,7.0,0.0,...,0.260216,0.112081,0.078082,0.062078,0.050540,-4.926690,1.001106,0.826562,0.138226,0.754033
150003,50688,20100405,0.0,0,0.0,0.0,1.0,160,7.0,0.0,...,0.260466,0.106727,0.081146,0.075971,0.048268,-4.864637,0.505493,1.870379,0.366038,1.312775
150004,161428,19970703,26.0,14,2.0,0.0,0.0,75,15.0,0.0,...,0.250999,0.000000,0.077806,0.028600,0.081709,3.616475,-0.673236,-3.197685,-0.025678,-0.101290


In [18]:
data_train.columns

Index(['name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox',
       'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller',
       'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13',
       'v_14'],
      dtype='object')

## 数据类型

In [21]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 30 columns):
name                 150000 non-null int64
regDate              150000 non-null int64
model                149999 non-null float64
brand                150000 non-null int64
bodyType             145494 non-null float64
fuelType             141320 non-null float64
gearbox              144019 non-null float64
power                150000 non-null int64
kilometer            150000 non-null float64
notRepairedDamage    150000 non-null object
regionCode           150000 non-null int64
seller               150000 non-null int64
offerType            150000 non-null int64
creatDate            150000 non-null int64
price                150000 non-null int64
v_0                  150000 non-null float64
v_1                  150000 non-null float64
v_2                  150000 non-null float64
v_3                  150000 non-null float64
v_4                  150000 non-null float64
v_5     

### 数值型变量
- `model`:车型编码是数值的？

In [25]:
list(data_train.dtypes[data_train.dtypes == "float64"].index)

['model',
 'bodyType',
 'fuelType',
 'gearbox',
 'kilometer',
 'v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11',
 'v_12',
 'v_13',
 'v_14']

In [26]:
data_train["model"].value_counts()

0.0      11762
19.0      9573
4.0       8445
1.0       6038
29.0      5186
         ...  
245.0        2
209.0        2
240.0        2
242.0        2
247.0        1
Name: model, Length: 248, dtype: int64